# Introdución a Tensorflow y Keras

Referencias:

- [Introducción a TensorFlow y Keras: Fundamentos y ejemplos](https://openwebinars.net/blog/tensorflow-keras-fundamentos/). OpenWebinars.
- [Introducción a las redes de memoria a corto-largo plazo (LSTM)](https://la.mathworks.com/discovery/lstm.html). MathWorks.
- [What Is a Recurrent Neural Network (RNN)?](https://la.mathworks.com/discovery/rnn.html). MathWorks.



## 1. Instalación de Tensorflow 2

- Necesitará un versión de Python que esté entre la **3.8 y [3.11](https://www.python.org/downloads/release/python-3119/)**.
- Cuando trabaje en su equipo de forma local se recomienda el uso [**entornos virtuales** con `virtualenv`](https://josejuansanchez.org/python-for-java-developers/#_entornos_virtuales).
- Descarga e instala el paquete de `tensorflow` con `pip`.



In [ ]:
!pip install tensorflow

## 2. Ejemplo de creación de un modelo LSTM

En este ejemplo vamos a construir un modelo **LSTM (_Long Short-Term Memory_)** para un Chatbot que será capaz de analizar una frase de un cliente y clasificarla en uno de las siguientes categorías:

- AYUDA
- OK
- SERVICIO_TECNICO


Los pasos que vamos a seguir son:

- Cargar un _dataset_ con los datos de entrenamiento. En nuestro caso será un archivo de texto.
- Construir un modelo de una red reuronal de tipo LSTM capaz de analizar texto.
- Entrenar la red neuronal.
- Evaluar la precisión del modelo.

### 2.1 Datos de entrenamiento

El archivo de texto que vamos a utilizar contiene frases con el siguiente formato:

```
Frase : CATEGORÍA
```

Ejemplo:


```
No tengo idea de qué hacer, necesito orientación : AYUDA
El equipo está funcionando sin problemas : OK
Necesito que un profesional venga a mi casa para inspeccionar el equipo : SERVICIO_TECNICO
```


In [ ]:
# Importamos Tensorflow y NumPy
import tensorflow as tf
import numpy as np
import json

"""
Función para cargar los datos de entrenamiento.
Esta función lee los datos de entrenamiento desde un archivo de texto
los almacena en dos listas: preguntas y respuestas, que luego devuevle
"""
def cargar_datos(nombre_archivo):
  preguntas = []
  respuestas = []
  with open(nombre_archivo, 'r') as file:
    #i = 1
    for line in file:
      pregunta, respuesta = line.strip().split(' : ')
      preguntas.append(pregunta)
      respuestas.append(respuesta)
      #print(f"{i} - {line}")
      #i += 1
  return preguntas, respuestas

"""
Función para preprocesar los datos.
Convierte las palabras en símbolos, para poder trabajar con ellas.
"""
def preprocesar_datos(preguntas, respuestas):
    # Creamos una capa de TextVectorization de TensorFlow, que nos ayuda
    # a convertir texto en representaciones numéricas (vectores o tokens).
    tokenizer = tf.keras.layers.TextVectorization()

    # El tokenizer "aprende" el vocabulario que se usa en las preguntas,
    # analiza su frecuencia y crea un mapeo único de palabras/tokens a índices numéricos.
    tokenizer.adapt(preguntas)

    # Las preguntas se convierten en secuencias numéricas usando el vocabulario aprendido.
    # Por ejemplo: "¿Cómo estás?" podría transformarse en [3, 15].
    x_train = tokenizer(preguntas)

    # Las respuestas se convierten en un array de NumPy, que es necesario
    # para la compatibilidad con TensorFlow/Keras durante el entrenamiento.
    y_train = np.array(respuestas)

    return x_train, y_train, tokenizer


"""
Función para construir el modelo LSTM (Long Short-Term Memory).
Este modelo está diseñado para procesar texto y clasificarlo en categorías.

El modelo es una red secuencial (Sequential) con tres capas clave:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(...),  # Capa 1: Embedding
    tf.keras.layers.LSTM(4),         # Capa 2: LSTM
    tf.keras.layers.Dense(3, ...)    # Capa 3: Salida
])

- Capa 1:
Convierte tokens numéricos (ej: [1, 14, 3]) en vectores densos de tamaño fijo.

- Capa 2:
Procesa secuencias de embeddings para capturar dependencias temporales como contexto en texto).
El número de unidades/células LSTM determina la capacidad de aprendizaje del modelo.
En este ejemplo se han utilizado 4 unidades/células.
Aumentar este valor (Ejemplo: 64) puede mejorar la capacidad del modelo,
pero requiere más datos para evitar overfitting.

- Capa 3:
Clasifica la secuencia en una de las 3 clases (definidas por num_clases).
"""
def construir_modelo(tokenizer, num_clases):
  # El modelo es una red secuencial (Sequential) con tres capas.
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(input_dim=len(tokenizer.get_vocabulary()) + 1, output_dim=4,  mask_zero=True),
      tf.keras.layers.LSTM(4), # La modificación de este parámetro redunda en la "inteligencia" del modelo
      tf.keras.layers.Dense(num_clases, activation='softmax')
  ])

  # Compilación del modelo.
  # Elegimos una función de pérdida (loss): sparse_categorical_crossentropy (adecuada para clases enteras, como 0, 1, 2).
  # Elegimos un optimizador (optimizer): adam (eficiente y popular para entrenamiento).
  # Métrica: accuracy (porcentaje de predicciones correctas).
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

"""
Función para entrenar el modelo.
- epoch: Es un ciclo completo de entrenamiento, donde el modelo ve todas las muestras del dataset una vez.
  El número de epochs se determina mediante validación (Ej: cuando la pérdida en la validación deja de mejorar).

- batch_size: Define cuántas muestras se procesan antes de actualizar los pesos del modelo.
  Podemos decir que controla lo fino del entrenamiento.

  Los valores de epoch y batch_size son responsables de que el entrenamiento tarde más o menos.

- verbose: Es la cantidad de información que muestra durante el entrenamiento
"""
def entrenar_modelo(model, x_train, y_train):
  model.fit(x_train, y_train, epochs=12, batch_size=1, verbose=True)

"""
Función para probar el chatbot
"""
def probar_chatbot(model, tokenizer, etiquetas):
  print("Chatbot: ¡Hola! Estoy listo para responder tus preguntas. Escribe 'salir' para finalizar.")

  while True:
    # Leemos una pregunta por teclado
    preguntab = input("Tú: ")
    if preguntab.lower() == "salir":
      break

    # Convierte la pregunta en una secuencia numérica (vector/token)
    pregunta = tokenizer([preguntab])

    # El modelo predice la categoría de la pregunta
    respuesta = model.predict(pregunta)
    print(respuesta)

    # Obtenemos el índice de la clase con mayor probabilidad según la predicción del modelo.
    indice = np.argmax(respuesta)
    print(indice)

    # Obtenemos el valor del índice. Necesitamos invertir el diccionario de etiquetas
    # Origen:  etiquetas = {"OK": 0, "AYUDA": 1, "SERVICIO_TECNICO": 2}
    # Destino: etiquetas_inversas = {0: "OK", 1: "AYUDA", 2: "SERVICIO_TECNICO"}
    etiquetas_inversas = {valor: clave for clave, valor in etiquetas.items()}
    categoria = etiquetas_inversas.get(indice, "NO ENTIENDO")

    # Esto se usa para añadir más muestras de entrenamiento al archivo y,
    # para que el modelo pueda aprender de ellas en un entrenamiento posterior.
    print("Chatbot: " + categoria)

    valido = input("¿Correcto? (S/N)")

    if valido.lower() == "s":
      # Nombre del archivo de texto donde están los datos de entrenamiento
      nombre_archivo = "tsetdesordenado.txt"

      # Abre el archivo en modo de escritura (append) para añadir contenido al final
      with open(nombre_archivo, "a") as archivo:
          # Escribe PREGUNTA, " : ", y CATEGORIA al final del archivo
          archivo.write(f"\n{preguntab} : {categoria}")

"""
Función para convertir las etiquetas de las categorías en índices.
Entrada:   etiquetas = {"OK": 0, "AYUDA": 1, "SERVICIO_TECNICO": 2}
           y_train = ["SERVICIO_TECNICO", "OK", "AYUDA", "OK", "AYDUDA", ...]
Salida:    y = [2, 0, 1, 0, 1, ...]
"""
def mapear_etiquetas_a_indices(y_train, etiquetas):
    # Convertimos las etiquetas de las categorías en índices
    y = [etiquetas[label] for label in y_train]

    # Convertimos la lista de enteros en un tensor de TensorFlow
    y = tf.convert_to_tensor(y, dtype=tf.int64)

    return y


# Programa Principal
if __name__ == "__main__":
    # Paso 1. Cargamos los datos de entrenamiento
    archivo_entrenamiento = "tsetdesordenado.txt"
    preguntas, respuestas = cargar_datos(archivo_entrenamiento)
    x_train, y_train, tokenizer = preprocesar_datos(preguntas, respuestas)

    # Convertimos el texto de las categorías de la lista y_train en índices.
    etiquetas = {"OK": 0, "AYUDA": 1, "SERVICIO_TECNICO": 2}
    y = mapear_etiquetas_a_indices(y_train, etiquetas)

    # Convertimos la lista de respuestas en un set para eliminar los elementos repetidos
    # Una vez que eliminamos los elementos repetidos contamos cuántos tipos hay.
    num_clases = len(set(respuestas))

    # Paso 2. Construimos el modelo
    model = construir_modelo(tokenizer, num_clases)

    # Paso 3. Entrenamos el modelo
    entrenar_modelo(model,  x_train, y)

    # Paso 4. Evaluamos el modelo
    probar_chatbot(model, tokenizer, etiquetas)

    # ----- Esto es una prueba -----
    # Guardar el modelo entrenado en formato Keras (o .h5)
    model.save("modelo.keras")

    # Guardar el vocabulario del tokenizer
    vocabulary = tokenizer.get_vocabulary()
    with open("vocabulary.txt", "w") as f:
        for word in vocabulary:
            f.write(f"{word}\n")

    # Guardar el mapeo de etiquetas en un archivo
    with open("etiquetas.json", "w") as f:
        json.dump(etiquetas, f)

Epoch 1/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6330 - loss: 0.9493
Epoch 2/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9955 - loss: 0.1393
Epoch 3/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9896 - loss: 0.0501
Epoch 4/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 1.0000 - loss: 0.0192
Epoch 5/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 1.0000 - loss: 0.0109
Epoch 6/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 1.0000 - loss: 0.0071
Epoch 7/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 1.0000 - loss: 0.0045
Epoch 8/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 0.0032
Epoch 9/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 1.0000 - loss: 0.0022
Epoch 10/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 11/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 0.0011
Epoch 12/12
570/570 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step

## Ejemplo de cómo utilizar un modelo ya entrenado

In [ ]:
import tensorflow as tf
import numpy as np
import json

# 1. Cargar el modelo
model = tf.keras.models.load_model("modelo.keras")

# 2. Cargar el vocabulario del tokenizer
with open("vocabulary.txt", "r") as f:
    vocabulary = [line.strip() for line in f]

# 3. Crear el tokenizer y asignar el vocabulario
tokenizer = tf.keras.layers.TextVectorization()
tokenizer.set_vocabulary(vocabulary)

# 4. Cargar el mapeo de etiquetas
with open("etiquetas.json", "r") as f:
    etiquetas = json.load(f)
    # Invertir el mapeo (de índice a etiqueta)
    index_to_label = {v: k for k, v in etiquetas.items()}

# Función para preprocesar texto de entrada
def preprocesar_texto(texto, tokenizer):
    texto_preprocesado = tokenizer([texto])
    return texto_preprocesado

# Bucle interactivo
print("Chatbot: ¡Hola! Escribe 'salir' para terminar.")
while True:
    entrada = input("Tú: ")
    if entrada.lower() == "salir":
        break

    # Preprocesar y predecir
    texto_preprocesado = preprocesar_texto(entrada, tokenizer)
    prediccion = model.predict(texto_preprocesado)
    indice = np.argmax(prediccion[0])  # Obtener el índice de mayor probabilidad

    # Obtener la etiqueta correspondiente
    etiqueta = index_to_label.get(indice, "NO ENTIENDO")
    print(f"Chatbot: {etiqueta}")

Chatbot: ¡Hola! Escribe 'salir' para terminar.
Tú: Todo está funcionando bien
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Chatbot: OK
Tú: Me gustaría hablar con alguien
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
Chatbot: AYUDA
Tú: Necesito que alguien venga a mi domicilio
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Chatbot: SERVICIO_TECNICO
Tú: salir


## Función auxiliar

Esta función nos ayuda a desordenar las líneas del archivo de datos de entrenamiento.

In [ ]:
import random

def barajar_lineas_archivo(input_file, output_file):
    """
    Baraja las líneas de un archivo de entrada y las guarda en un archivo de salida.

    Parámetros:
    input_file (str): Ruta del archivo de entrada
    output_file (str): Ruta del archivo de salida
    """
    # Leer todas las líneas del archivo de entrada
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Barajar las líneas
    random.shuffle(lines)

    # Escribir las líneas barajadas en el archivo de salida
    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(lines)

In [ ]:
barajar_lineas_archivo("tset.txt", "tsetdesordenado.txt")